In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import os
from os import listdir

In [ ]:
os.chdir(r'd:\Documentos\NOVENO SEMESTRE\Pf\Ltsm\MovementAAL\dataset')
path = 'MovementAAL_RSS_'

#Sequences es la lista de los vectores con los datos de cada csv
#314 csv

sequences = list()
for i in range(1,315):
    file_path = path + str(i) + '.csv'
    # print(file_path)
    df = pd.read_csv(file_path, header=0)
    values = df.values
    sequences.append(values)
print(sequences)

In [ ]:
os.chdir(r'd:\Documentos\NOVENO SEMESTRE\Pf\Ltsm\MovementAAL\dataset')
targets = pd.read_csv('MovementAAL_target.csv')
targets = targets.values[:,1]
print(targets)
#Le da un objetivo a cada csv

In [ ]:
os.chdir(r'd:\Documentos\NOVENO SEMESTRE\Pf\Ltsm\MovementAAL\groups')
groups = pd.read_csv('MovementAAL_DatasetGroup.csv', header=0)
groups = groups.values[:,1]
print(groups)

In [ ]:
#contar muestras de cada secuencia
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
    #print(len_sequences)
series=pd.Series(len_sequences)
series.describe()

In [ ]:
#Padding the sequence with the values in last row to max length
to_pad = 129
new_seq = []
#Se trabaja sobre cada csv iterando el vector secuencias
for one_seq in sequences:
    #Obtiene la cantidad de datos de cada csv
    len_one_seq = len(one_seq)
    #Busca los ultimos valores del csv
    last_val = one_seq[-1]
    #Resta a la cantidad de datos del csv, el valor maximo de los que hay
    #Esto para saber cuantos datos debe reproducir y alcanzar los 129
    n = to_pad - len_one_seq
   
    to_concat = np.repeat(one_seq[-1], n).reshape(4, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)

In [29]:
#truncate the sequence to length 60
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

#from tf.keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

seq_len = 60
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')
type(final_seq)


numpy.ndarray

In [30]:
train = [final_seq[i] for i in range(len(groups)) if (groups[i]==2)]
validation = [final_seq[i] for i in range(len(groups)) if groups[i]==1]
test = [final_seq[i] for i in range(len(groups)) if groups[i]==3]

train_target = [targets[i] for i in range(len(groups)) if (groups[i]==2)]
validation_target = [targets[i] for i in range(len(groups)) if groups[i]==1]
test_target = [targets[i] for i in range(len(groups)) if groups[i]==3]

In [31]:
train = np.array(train)
validation = np.array(validation)
test = np.array(test)

train_target = np.array(train_target)
train_target = (train_target+1)/2

validation_target = np.array(validation_target)
validation_target = (validation_target+1)/2

test_target = np.array(test_target)
test_target = (test_target+1)/2

In [33]:
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, 4)))
model.add(Dense(1, activation='sigmoid'))
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 256)               267264    
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 267,521
Trainable params: 267,521
Non-trainable params: 0
_________________________________________________________________
